In [1]:
from pyspark.ml.feature import StringIndexer

In [2]:
df = spark.read.load("sample/test.csv",
                     format="csv", sep=",", inferSchema="true", header="false")
df = df.toDF('subreddit', 'author', 'id',
'author_flair_css_class',
'author_flair_text',
'controversiality',
'created_utc',
'distinguished',
'edited',
'gilded',
'is_submitter',
'link_id',
'parent_id',
'permalink',
'retrieved_on',
'score',
'stickied',
'subreddit_id',
'subreddit_type',
'body')
df = df.select(df['subreddit'], df['body'])

In [3]:
res = df.rdd.map(lambda r: (r[0], r[1] if not r[1] is None else "" )).groupByKey().map(lambda e: (e[0], " ".join(e[1])))

In [4]:
res_df = res.toDF().toDF('sub', 'words')
res_df.write.csv('sub_words.csv')

In [ ]:
from pyspark.ml.feature import RegexTokenizer
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import HashingTF, IDF

tkn = RegexTokenizer().setInputCol("words").setOutputCol("subwords").setPattern("\\W").setToLowercase(True)
tokenized = tkn.transform(res_df)

englishStopWords = StopWordsRemover.loadDefaultStopWords("english")
stops = StopWordsRemover().setStopWords(englishStopWords).setInputCol("subwords").setOutputCol('without_stops')

stoprem = stops.transform(tokenized)
stoprem.select('sub', 'without_stops').rdd.map(lambda r: (r[0], " ".join(r[1]))).toDF().write.csv('stopsrem.csv')

"""
tmp = stoprem.select('without_stops')

tf = HashingTF().setInputCol("without_stops").setOutputCol("TFOut").setNumFeatures(1000)
idf = IDF().setInputCol("TFOut").setOutputCol("IDFOut").setMinDocFreq(2)

idf.fit(tf.transform(tmp)).transform(tf.transform(tmp)).show(2, False)
"""

In [14]:
sub_indexer = StringIndexer(inputCol='subreddit', outputCol='subindex') 
indexed = sub_indexer.fit(df).transform(df)

In [18]:
indexed.rdd.map(lambda x: x).collect()

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job 10 cancelled because SparkContext was shut down
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:837)
	at org.apache.spark.scheduler.DAGScheduler$$anonfun$cleanUpAfterSchedulerStop$1.apply(DAGScheduler.scala:835)
	at scala.collection.mutable.HashSet.foreach(HashSet.scala:78)
	at org.apache.spark.scheduler.DAGScheduler.cleanUpAfterSchedulerStop(DAGScheduler.scala:835)
	at org.apache.spark.scheduler.DAGSchedulerEventProcessLoop.onStop(DAGScheduler.scala:1841)
	at org.apache.spark.util.EventLoop.stop(EventLoop.scala:83)
	at org.apache.spark.scheduler.DAGScheduler.stop(DAGScheduler.scala:1754)
	at org.apache.spark.SparkContext$$anonfun$stop$8.apply$mcV$sp(SparkContext.scala:1931)
	at org.apache.spark.util.Utils$.tryLogNonFatalError(Utils.scala:1360)
	at org.apache.spark.SparkContext.stop(Spark